In [1]:
import numpy as np

import pandas as pd

import warnings
warnings.filterwarnings('ignore')

## Conjunto de Dados
Lê o conjunto de dados de treino e teste.

In [2]:
train = pd.read_csv("data/reviews_train.csv")
test = pd.read_csv("data/reviews_test.csv")

## Comente as diferenças entre as bases de dados

In [3]:
train.head()

,id,sentiment,review
0,1570,1,We have been going here for about 3 years now ...
1,3294,0,Dog Park Who are they kidding This place was c...
2,1727,1,Loves me some Trader Joe'sOnly disappointment ...
3,3362,0,Given the option of being punched in the face ...
4,3204,0,Atrocious Consider the price point Dollar for ...


In [4]:
test.head()

,id,review
0,1197,Egg drop soup at ninety-nine places out of one...
1,3857,The Vue has changed management companiesnow a ...
2,1949,Wonderful country store Christmas time was nic...
3,2080,Carolina's has the best tortillas in town hand...
4,660,I can also vouch for this incredible veterinar...


## Análise de Sentimentos

Nós estamos interessados em predizer o sentimento das revisões a partir de seu contéudo. Portanto, devemos construir um modelo que seja capaz de classificar o sentimento (positivo ou negativo) usando o texto.

### TODO: Abra a documentação do Scikit-learn e faça testes sobre os hiperparâmetros
http://scikit-learn.org/stable/modules/generated/sklearn.tree.DecisionTreeClassifier.html

In [5]:
from sklearn.metrics import accuracy_score
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import StratifiedShuffleSplit
from sklearn.feature_extraction.text import TfidfVectorizer

# Refere-se ao número de partições que serão utilizadas para avaliar o modelo
n_splits=5

# Configurando variável para separar dados de treino e teste
sss = StratifiedShuffleSplit(n_splits=n_splits, test_size=0.2, random_state=0)

# Extrator de features baseadas no texto
# O extrator deve considerar o texto de todas as reviões (tanto do treino quanto teste)
tfidf = TfidfVectorizer(stop_words = 'english', norm='l2')

# Processa as features
tfidf.fit(np.append(train["review"].values, test["review"].values))

# Recupera os dados separados para treino
X = tfidf.transform(train["review"].values)
y = train["sentiment"].values

# Declara o modelo que será utilizado para classificação
model = DecisionTreeClassifier()

id_fold = 0

# Salva os resultados de precisão para cada processo de treino/teste executado
acc_average = np.zeros(n_splits * 2).reshape(n_splits, 2)

# Particiona a base de dados em k
# Mesmo sobre a base de treino, os dados devem ser particionados para validação dos melhores modelos
for train_index, test_index in sss.split(X, y):
    id_fold += 1
    
    # Treino
    pred = model.fit(X[train_index], y[train_index]).predict(X[train_index])
    
    # Predição sobre os dados de treino
    print("Partição", id_fold)
    acc_average[id_fold -1, 0] = accuracy_score(y[train_index], pred, normalize=True)
    print("  Acurácia no treino", acc_average[id_fold -1, 0])

    #Predição sobre os dados de teste
    pred = model.predict(X[test_index])
    acc_average[id_fold -1, 1] = accuracy_score(y[test_index], pred, normalize=True)
    print("  Acurácia no teste", acc_average[id_fold -1, 1])

# Resultado médio
print("\nAcurácia média no treino é", np.mean(acc_average[:, 0]), "e no teste é", np.mean(acc_average[:, 1]))

Partição 1
  Acurácia no treino 0.999642857143
  Acurácia no teste 0.785714285714
Partição 2
  Acurácia no treino 1.0
  Acurácia no teste 0.781428571429
Partição 3
  Acurácia no treino 1.0
  Acurácia no teste 0.802857142857
Partição 4
  Acurácia no treino 0.999642857143
  Acurácia no teste 0.76
Partição 5
  Acurácia no treino 1.0
  Acurácia no teste 0.774285714286

Acurácia média no treino é 0.999857142857 e no teste é 0.780857142857


## Gerando arquivo de saída para submissão no kaggle

In [6]:
X_test = tfidf.transform(test["review"].values)
test["sentiment"] = model.predict(X_test)
test[["id", "sentiment"]].to_csv("data/submission.csv", index=False)

<center><h1>========================== END ==========================</h1></center>